In [1]:
from sklearn.datasets import load_boston

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Loading and Examining Initial Dataset

In [3]:
boston = load_boston()
print(boston.data.shape)

(506, 13)


In [ ]:
boston.target.shape

In [ ]:
print(boston.DESCR)

### Step 1: Setting up Initial Regression

In [4]:
x = boston.data[:,np.r_[0:11,12]]
y = boston.target

In [ ]:
x.shape

In [5]:
def regTesting(x,y):
    regr = LinearRegression()
    regr.fit(x,y)
    y_pred = regr.predict(x)
    mse = mean_squared_error(y,y_pred)
    r2 = r2_score(y,y_pred)
    return mse, r2

In [6]:
base_mse, base_r2 = regTesting(x,y)
print('Baseline MSE:', base_mse,'\nBaseline R2:', base_r2)

Baseline MSE: 22.429681439489926 
Baseline R2: 0.7343070437613076


### Step 2: MCAR Imputation Performance

In [ ]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

In [ ]:
perc_array = [0.01, 0.05, 0.10, 0.2, 0.33, 0.5]

In [ ]:
def mcar_dataset(x, rate, random = 44):
    rng = np.random.RandomState(random)
    missing_rate = rate
    length = len(x)
    num_to_remove = int(np.floor(length * missing_rate))
    missing_records_bool = np.hstack((np.zeros(length - num_to_remove,
                                          dtype=np.bool),
                                 np.ones(num_to_remove,
                                         dtype=np.bool)))
    rng.shuffle(missing_records_bool)
    
    X_missing = x.copy()

    # change the [0] index to represent the column to be reduced
    X_missing[missing_records_bool,0] = np.nan
    return X_missing

In [ ]:
# The sample_posterior = True is to allow for multiple imputation, if we're doing that. Would need to modify the random_state for each iteration (maybe just remove the random_state?)
imputer = IterativeImputer(missing_values=np.nan, sample_posterior=True, random_state=0)

In [ ]:
imputed_scores = []
for rate in perc_array:
    x_new = mcar_dataset(x,rate)
    x_impute = imputer.fit_transform(x_new)
    mse, r2 = regTesting(x_impute,y)
    imputed_scores.append((rate, mse,r2))

In [ ]:
imputed_scores

In [ ]:
dt=np.dtype('float,float,float')
imp_scores_arr = np.array(imputed_scores,dtype=dt)
imp_scores_arr.dtype.names=['Missing Rate','MSE','R2']
imp_scores_arr

In [7]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(12, 6))
ax1 = plt.subplot(121)
ax1.plot(imp_scores_arr['Missing Rate'], imp_scores_arr['MSE'], alpha=0.6, c='blue', marker="o")
ax1.plot(0, base_mse, 'gs')
ax1.set_title('MSE of Imputed Missing Data')
ax1.set_xlim(left=imp_scores_arr['Missing Rate'][0]*-1, right=imp_scores_arr['Missing Rate'][5]* 1.1)
ax1.set_xlabel('Missing Rate')

ax2 = plt.subplot(122)
ax2.plot(imp_scores_arr['Missing Rate'], imp_scores_arr['R2'], alpha=0.6, c='blue', marker="o")
ax2.plot(0,base_r2, 'gs')
ax2.set_title('Goodness of Fit of Imputed Missing Data')
ax2.set_xlim(left=imp_scores_arr['Missing Rate'][0]*-1, right=imp_scores_arr['Missing Rate'][5]* 1.1)
ax2.set_xlabel('Missing Rate')

plt.show()

### Step 3: MAR Imputation Performance

Take 2 different columns and create data “Missing at Random” when controlled for a third variable (i.e if Variable Z is > 30, than Variables X, Y are randomly missing).  Make runs with 10%, 20% and 30% missing data imputed via your best guess.  Repeat your fit and comparisons to the baseline.

In [ ]:
len(np.where(x[:,4]>(0.6*max(x[:,4])))[0])

In [ ]:
perc_array2 = [0.10, 0.2, 0.3]

In [ ]:
def mar_dataset(x,rate,random=0):
    np.random.seed(random)  # set the seed
    # creates a boolean that looks for where the NO levels are greater than 60% of the maximum value of NO in the dataset
    x_check = np.where(x[:,4]>(0.6*max(x[:,4])))
    print("x_check length is: {0}".format(len(x_check[0])))
    num = int(np.floor(rate*len(x_check[0])))
    print("num NaN'd: {0}".format(num))
    missing = np.random.choice(x_check[0], num ,replace=False)
    X_missing = x.copy()
    X_missing[missing, 5:6] = np.nan # introduces NaNs into RM and AGE
    return X_missing

In [ ]:
imputed_scores2 = []
for rate in perc_array2:
    x_new = mar_dataset(x,rate)
    x_impute = imputer.fit_transform(x_new)
    mse, r2 = regTesting(x_impute,y)
    imputed_scores2.append((rate, mse,r2))

In [ ]:
dt2=np.dtype('float,float,float')
imp_scores_arr2 = np.array(imputed_scores2,dtype=dt2)
imp_scores_arr2.dtype.names=['Missing Rate','MSE','R2']
imp_scores_arr2

In [ ]:
plt.figure(figsize=(12, 6))
ax3 = plt.subplot(121)
ax3.plot(imp_scores_arr2['Missing Rate'], imp_scores_arr2['MSE'], alpha=0.6, c='blue', marker="o")
ax3.plot(0, base_mse, 'gs')
ax3.set_title('MSE of Imputed Missing Data')
ax3.set_xlim(left=-.01, right=imp_scores_arr2['Missing Rate'][2]* 1.1)
ax3.set_xlabel('Missing Rate')

ax4 = plt.subplot(122)
ax4.plot(imp_scores_arr2['Missing Rate'], imp_scores_arr2['R2'], alpha=0.6, c='blue', marker="o")
ax4.plot(0,base_r2, 'gs')
ax4.set_title('Goodness of Fit of Imputed Missing Data')
ax4.set_xlim(left=-.01, right=imp_scores_arr2['Missing Rate'][2]* 1.1)
ax4.set_xlabel('Missing Rate')

plt.show()

### Step 4: MNAR Imputation Performance 
Create a Missing Not at Random pattern in which 25% of the data is missing for a single column. 

In [10]:
# To find what value the highest 25% of CRIM are (to know where to slice)
np.quantile(x[:,0],.75)

3.6770825

In [11]:
len(np.where(x[:,0]>np.quantile(x[:,0],.75))[0])

127

In [12]:
def mnar_dataset(x,rate = 0.25,random=0):
    rate = 1-rate
    # creates a boolean that looks for where the CRIM levels are greater than 75th percentile
    x_check = np.where(x[:,0]>np.quantile(x[:,0],rate))
    X_missing = x.copy()
    X_missing[x_check, 0] = np.nan
    return X_missing, x_check

In [14]:
x_new, x_check = mnar_dataset(x)



In [ ]:
x_impute = imputer.fit_transform(x_new)
mse_mnar, r2_mnar = regTesting(x_impute,y)

In [ ]:
x_check

In [117]:
print('Baseline MSE:', base_mse,'\nBaseline R2:', base_r2)
print('\nMNAR Imputed MSE:', mse_mnar,'\nMNAR Imputed R2:', r2_mnar)

Baseline MSE: 22.429681439489926 
Baseline R2: 0.7343070437613076

MNAR Imputed MSE: 78.30022904824078 
MNAR Imputed R2: r2_r(r2_median=0.07, r2_mean=0.07, r2_std=0.0)


### Step 5 (Extra Credit): MCMC Imputation Performance
Using the MCMC method, and your data from step 4, What is the difference in performance between imputation via ‘guess’ (mean/median, etc) and MCMC.

In [18]:
#  https://towardsdatascience.com/markov-chain-monte-carlo-in-python-44f7e609be98
#  https://docs.pymc.io/notebooks/getting_started.html

from pymc3 import *

In [102]:
# create matrix of rows with missing data in CRIM
X_missing_subset = x_new[x_check,:]

# create matrix of rows that are complete (dropping rows that are missing value for CRIM)
mask = np.ones(len(x_new), np.bool)
mask[x_check] = 0
X_whole_subset = x_new[mask]

In [122]:
X1 = X_whole_subset[:,1] 
X2 = X_whole_subset[:,2]
X3 = X_whole_subset[:,3]
X4 = X_whole_subset[:,4]
X5 = X_whole_subset[:,5]
X6 = X_whole_subset[:,6]
X7 = X_whole_subset[:,7]
X8 = X_whole_subset[:,8]
X9 = X_whole_subset[:,9]
X10 = X_whole_subset[:,10]
X11 = X_whole_subset[:,11]


Y = X_whole_subset[:,0]

data = dict(X1=X1, X2=X2, X3=X3, X4=X4, X5=X5, X6=X6, X7=X7, X8=X8, X9=X9, X10=X10, X11=X11, Y=Y)

In [123]:
MCMCModel = pm.Model()

with MCMCModel:
    glm.GLM.from_formula('Y ~ X1 + X2 + X3 + X4 + X5 + X6 + X7 + X8 + X9 + X10 + X11', data)
    trace = sample(3000, cores=2) # draw 3000 posterior samples using NUTS sampling

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sd, X11, X10, X9, X8, X7, X6, X5, X4, X3, X2, X1, Intercept]
Sampling 2 chains: 100%|██████████| 7000/7000 [14:52<00:00,  3.04draws/s]
The acceptance probability does not match the target. It is 0.952106968346552, but should be close to 0.8. Try to increase the number of tuning steps.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
The acceptance probability does not match the target. It is 0.9287307180615588, but should be close to 0.8. Try to increase the number of tuning steps.
The gelman-rubin statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.


In [124]:
parameters = pm.find_MAP(model=MCMCModel)

/Users/benjaminwilke/anaconda3/lib/python3.7/site-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
logp = -287.14, ||grad|| = 7.7845: 100%|██████████| 2032/2032 [00:01<00:00, 1078.59it/s]  


In [74]:
parameters

{'Intercept': array(-2.00857897),
 'X1': array(0.0020852),
 'X2': array(0.01252408),
 'X3': array(0.14025759),
 'X4': array(4.43131502),
 'sd_log__': array(-0.70780317),
 'sd': array(0.49272544)}

In [125]:
params = {"inter": parameters["Intercept"].item(0), "X1": parameters["X1"].item(0),
         "X2": parameters["X2"].item(0), "X3": parameters["X3"].item(0),
         "X4": parameters["X4"].item(0), "X5": parameters["X5"].item(0), 
         "X6": parameters["X6"].item(0), "X7": parameters["X7"].item(0), 
         "X8": parameters["X8"].item(0), "X9": parameters["X9"].item(0), 
         "X10": parameters["X10"].item(0), "X11": parameters["X11"].item(0), 
         }

In [126]:
params

{'inter': -1.9266196530429902,
 'X1': 0.0006671423503375285,
 'X2': 0.014673089238562857,
 'X3': 0.06744226665929479,
 'X4': 4.259807875394496,
 'X5': -0.017128154990272423,
 'X6': 4.543309339669889e-05,
 'X7': 0.047114415680519474,
 'X8': 0.09189783211692451,
 'X9': 4.7146169035044756e-05,
 'X10': -0.031076794271239452,
 'X11': 0.0016076343131654156}

In [129]:
for each in range(127):
    X_missing_subset[0][each][0] = params["inter"] + (X_missing_subset[0][each][1] * params["X1"]) + \
    (X_missing_subset[0][each][2] * params["X2"]) + (X_missing_subset[0][each][3] * params["X3"]) +  \
    (X_missing_subset[0][each][4] * params["X4"]) + (X_missing_subset[0][each][5] * params["X5"]) + \
    (X_missing_subset[0][each][6] * params["X6"]) + (X_missing_subset[0][each][7] * params["X7"]) + \
     (X_missing_subset[0][each][8] * params["X8"]) + (X_missing_subset[0][each][9] * params["X9"]) + \
    (X_missing_subset[0][each][10] * params["X10"]) + (X_missing_subset[0][each][11] * params["X11"])


In [108]:
X_missing_subset[0].shape

(127, 12)

In [105]:
X_whole_subset.shape

(379, 12)

In [130]:
new = np.concatenate((X_missing_subset[0], X_whole_subset), axis=0)

In [118]:
new.shape

(506, 12)

In [131]:
mse_mnar, r2_mnar = regTesting(new,y)

In [132]:
print('Baseline MSE:', base_mse,'\nBaseline R2:', base_r2)
print('\nMNAR Imputed MSE:', mse_mnar,'\nMNAR Imputed R2:', r2_mnar)

Baseline MSE: 22.429681439489926 
Baseline R2: 0.7343070437613076

MNAR Imputed MSE: 78.5247492188646 
MNAR Imputed R2: r2_r(r2_median=0.07, r2_mean=0.07, r2_std=0.0)
